In [6]:
# Loading-in our data!
import torchvision


# downloading 50k image net samples.
data = torchvision.datasets.Imagenette(
    root = "imagenette_download",
    split = "train",
    download=False
)

In [7]:
# using our extractor.
from dataloader import raw_images_convert
raw_images_convert(data,'raw_images','jpeg') 

# now, we can delete the real images, but keep for now.

Path exists, process skipped


In [ ]:
# now, we need to create our latent-model
from VAE import # LATENT-MODEL
from make_latents import store_latents

store_latents(data,'latent_storage',#LATENT_MODEL)